In [2]:
import numpy as np
from scipy import stats
import time
import datetime
import pandas as pd

import mysql.connector
from mysql.connector import Error

from bokeh.plotting import *
from bokeh.layouts import gridplot
from bokeh.models import *# Span, ColumnDataSource, LogColorMapper, ColorMapper, LogTicker, ColorBar, BasicTicker, LinearColorMapper, PrintfTickFormatter, HoverTool, CategoricalColorMapper, Range1d, Title
from bokeh.models.widgets import Tabs, Panel
from bokeh.io import show, output_notebook, reset_output
output_notebook()
from bokeh.models.glyphs import Text
import bokeh.palettes as bp
from bokeh.transform import factor_cmap

# import hail as hl
import json
import urllib
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)

from pathlib import Path
from joblib import Parallel, delayed

Loading BokehJS ...

In [3]:
from hv_setup import *

# Movie length histogram

In [ ]:
def bin_width(m):
    n = np.int(np.log10(m+1))
    n = 10**(n-1)
    q = np.ceil(m/(n*(36))).astype(int)
    bw = max(q*n,1)
    return bw#, m//n+1

In [ ]:
hv={}
# query="SELECT dataSourceString FROM movies"
# query = "SELECT DATE_FORMAT(reqStartDate, '%1980-%m-%d %H:%i:%S') AS reqStartDate, timestamp, reqEndDate, startDate, endDate, dataSourceString FROM movies"
# date_format(reqStartDate, '%Y-%m-%d %H:%i:%s') AS REQ_START, date_format(reqEndDate, '%Y-%m-%d %H:%i:%s')
query = "SELECT reqStartDate, reqEndDate, dataSourceString, eventSourceString, numFrames, frameRate, maxFrames, timestamp as date, TIMESTAMPDIFF(second, reqStartDate, reqEndDate) AS reqDuration, TIMESTAMPDIFF(second, startDate, endDate) AS genDuration FROM movies WHERE reqEndDate!='None' AND reqStartDate!='None' AND startDate!='None' AND endDate!='None';"
# query = "SELECT ROUND(TIMESTAMPDIFF(second, reqStartDate, reqEndDate)/60/60/24, 3) AS reqDuration, ROUND(TIMESTAMPDIFF(second, startDate, endDate)/60/60/24, 3) AS genDuration FROM movies;"
hv['movies'] = sql_query(query)

In [ ]:
df = hv['movies'].copy()

df['reqDuration'] = pd.to_timedelta(df['reqDuration'], unit='s')/pd.Timedelta(days=1)
df['reqDuration'].loc[df['reqDuration']>30] = np.nan
df['genDuration'] = pd.to_timedelta((df['numFrames']/df['frameRate']), unit='s')/pd.Timedelta(seconds=1)
outlier_count = df['genDuration'].loc[df['genDuration']>300]
outlier_date = df['date'].loc[df['genDuration']>300].dt.strftime("%b %d %Y, %H:%M:%S").values[0]
df['genDuration'].loc[df['genDuration']>300] = np.nan
df.sort_values('genDuration')

In [ ]:
key='movies'
# bin_size = 100# 0.5*24*60*60# np.arange(0,count.max(),) 30#.astype(int)#100
panels_pov=[]
for pov, ref, bin_size, unit in zip(['reqDuration','genDuration'], 
                                    ['requested','generated'],
                                    [0.5, 10],
                                    ['days','seconds']):

    counts = df[pov]

    arr_hist, edges = np.histogram(counts, bins=np.arange(0, counts.max()+bin_size, bin_size))
    cum_bin_size = max(bin_size//10, 1)
    cum_hist, cum_edges, patches = plt.hist(counts, bins=np.arange(0,counts.max()+cum_bin_size, cum_bin_size), cumulative=True)
    plt.close()

    # Column data source
    df_hist = pd.DataFrame({'count': arr_hist, 'left': edges[:-1], 'right': edges[1:]})
    total = df_hist['count'].sum()
    df_hist['f_count'] = ['%d' % count for count in df_hist['count']]
    df_hist['f_percent'] = ['%.3f%%' %(count/total*100) for count in df_hist['count']]
    df_hist['f_interval'] = ['[%.1f %s,%.1f %s)' % (left, unit, right, unit) for left, right in zip(df_hist['left'], df_hist['right'])]
    hist_src = ColumnDataSource(df_hist)

    #cumulative data
    cumulative_data = cum_hist#np.cumsum(arr_hist)
    x_bins = cum_edges[1:]#edges[1:]# np.arange(0, counts.max(), bin_size)[1:]
    df_cum = pd.DataFrame({'count_cum': cumulative_data, 'x': x_bins})
    df_cum['f_percent'] = ['%.3f%%' %(count/total*100) for count in df_cum['count_cum']]
    cum_src = ColumnDataSource(df_cum)

    panels = []
    for axis_type in ["log","linear"]:
        p = figure(y_axis_type = axis_type,
                   x_axis_label = 'Length of %s (%s)'%(key, unit), y_axis_label = 'Movie count', 
                   background_fill_color="#fafafa",
                   y_range = (0.5, df_hist['count'].max() + df_hist['count'].max()//10)
                  )

        # Add a quad glyph with source this time
        p_hist = p.quad(bottom=0.5, top='count', left='left', right='right', source=hist_src, fill_color='navy', alpha=0.5,
               hover_fill_color='navy', hover_fill_alpha=0.2, line_color='white', legend='Histogram')
        # Add style to the plot
        p.title.align = 'center'
        p.title.text_font_size = '18pt'
        p.xaxis.axis_label_text_font_size = '12pt'
        p.xaxis.major_label_text_font_size = '12pt'
        p.yaxis.axis_label_text_font_size = '12pt'
        p.yaxis.major_label_text_font_size = '12pt'
#         p.yaxis[0].formatter = PrintfTickFormatter(format="%f")
        p.yaxis[0].formatter = NumeralTickFormatter(format='0,0')
        
    #         p.add_layout(Span(location=1800, dimension='height'))#, legend='Expected date file count'))
        df_stats = pd.DataFrame({'height': np.linspace(min(df_hist['count'].min(),0.5),df_hist['count'].max(),2),
                                 'mean':np.nanmean(counts), 'median': np.nanmedian(counts), 'mode':stats.mode(counts)[0][0]})
        p.line(x='mean', y='height', line_color="black", line_dash='solid', line_width = 4, legend="Mean (%.2f %s)"%(df_stats['mean'][0], unit), source=df_stats)
        p.line(x='median', y='height', line_color = "red", line_dash='dashed', line_width=3, legend="Median (%.2f %s)"%(df_stats['median'][0],unit), source=df_stats)
        p.line(x='mode', y='height', line_color = "lightgreen", line_dash = 'dashdot',line_width=2,legend="Mode (%.2f %s)"%(df_stats['mode'][0],unit), source=df_stats)

        total_days = len(counts)
        total_files = counts.sum()

        p.add_layout(Title(text = "Histogram for length of %s %s"%(key, ref), text_font_size = "16pt", text_font_style="bold"), 
                     place = 'above')
        p.add_layout(Title(text="%s during: %s - %s"%(ref, df['date'].min().strftime('%Y, %b %d'),df['date'].max().strftime('%Y, %b %d'))), 
                     place = 'above')
        if(ref=='generated'):
            p.add_layout(Title(text="(Movie of length %d seconds %s on %s was discarded)"%(outlier_count, ref, outlier_date), text_font_style="italic"), 
                          place = 'above')
        p.add_layout(Title(text="Total length of {} {} ({}): {:,.2f} | Total Movies: {:,} ".format(key, ref, unit, total_files, total_days), text_font_style="italic"), 
                     place = 'above')

        p.legend.location = "top_right"
        p.grid.grid_line_color="white"

        # Add a hover tool referring to the formatted columns
        hover = HoverTool(tooltips = [('Length of %s %s'%(key, ref), '@f_interval'),
                                      ('Movie count', '@f_count{0,0}'),
                                      ('Movie count percentage', '@f_percent')],
#                           formatters={'f_count'      : 'printf', # use 'datetime' formatter for 'date' field
#                         'count' : 'int',   # use 'printf' formatter for 'adj close' field
#                                           use default 'numeral' formatter for other fields
#                                      },
                          mode= 'vline')

        # Add the hover tool to the graph
        p.add_tools(hover)
        p2 = figure(y_axis_type=axis_type,
                           x_axis_label = 'Length of %s (%s)'%(key, unit), 
                           y_axis_label = 'Movie count',
                           background_fill_color="#fafafa")


        p2_line = p2.line(x='x', y='count_cum', line_color='#036564', line_width=3, source=cum_src, legend="Cumulative distribution")
    #         p2_circle = p2.circle(x='x', y='count_cum', line_color='#036564', line_width=5, source=cum_src, hover_line_alpha=0.5, legend="Cumulative distribution" )
        p2.add_layout(Title(text = "Cumulative distribution for length of %s %s"%(key, ref), text_font_size = "16pt", text_font_style="bold"), 
                      place = 'above')
        p2.add_layout(Title(text="%s during: %s - %s"%(ref, df['date'].min().strftime('%Y, %b %d'),df['date'].max().strftime('%Y, %b %d'))), 
                      place = 'above')
        if(ref=='generated'):
            p2.add_layout(Title(text="(Movie of length %d seconds %s on %s was discarded)"%(outlier_count, ref, outlier_date), text_font_style="italic"), 
                          place = 'above')
        p2.add_layout(Title(text="Total length of {} {} ({}): {:,.2f} | Total Movies: {:,} ".format(key, ref, unit, total_files, total_days), text_font_style="italic"), 
                      place= 'above')

        hover = HoverTool(line_policy='nearest', 
                          tooltips = [('Length of %s %s'%(key, ref), '<@x{0.2f} %s'%unit), 
                                      ('Percentage of %s %s'%(key,ref), '<@f_percent'),
                                      ('Cumulative Day count', '@count_cum{0,0}')],
                          mode='vline')

        df_cumstats = pd.DataFrame({'height': np.linspace(df_cum['count_cum'].min(),df_cum['count_cum'].max(),2),
                                 'mean':np.nanmean(counts), 'median': np.nanmedian(counts), 'mode':stats.mode(counts)[0][0]})
        p2.line(x='mean', y='height', line_color="black", line_dash='solid', line_width = 4, legend="Mean (%.2f %s)"%(df_cumstats['mean'][0], unit), source=df_cumstats)
        p2.line(x='median', y='height', line_color = "red", line_dash='dashed', line_width=3, legend="Median (%.2f %s)"%(df_cumstats['median'][0], unit), source=df_cumstats)
        p2.line(x='mode', y='height', line_color = "lightgreen", line_dash = 'dashdot',line_width=2,legend="Mode (%.2f %s)"%(df_cumstats['mode'][0], unit), source=df_cumstats)
#         p2.add_layout(Span(location=10, dimension='height', legend='Expected date file count'))
        
        # Add the hover tool to the graph
        p2.add_tools(hover)
        p2.title.align = 'center'
        p2.title.text_font_size = '18pt'
        p2.xaxis.axis_label_text_font_size = '12pt'
        p2.xaxis.major_label_text_font_size = '12pt'
        p2.yaxis.axis_label_text_font_size = '12pt'
        p2.yaxis.major_label_text_font_size = '12pt'
        p2.legend.location = "bottom_right"
#         p2.yaxis[0].formatter = PrintfTickFormatter(format="0,0%f")
        p2.yaxis[0].formatter = NumeralTickFormatter(format='0,0')
        

        grid = gridplot([[p, p2]], sizing_mode='stretch_both')# width_policy='max', height_policy='max')#,plot_width=1200, plot_height=1000, sizing_mode='scale_width')#, plot_width=250, plot_height=250)
        panel = Panel(child=grid, title=axis_type)
        panels.append(panel)
    tabs = Tabs(tabs=panels)
    panel_pov = Panel(child=tabs, title=ref)
    panels_pov.append(panel_pov)

tabs_pov = Tabs(tabs=panels_pov)
show(tabs_pov)
save(tabs_pov, filename='./%s/histogram_length.html'%key, title='Histogram for length of Helioviewer %s'%key)

# Time series

In [ ]:
hv={}
hv['movies'] = sql_query("SELECT date_format(timestamp, '%Y-%m-%d 00:00:00') as date, count(*) as count FROM movies GROUP BY date_format(timestamp, '%Y-%m-%d 00:00:00');")
hv['screenshots'] = sql_query("SELECT date_format(timestamp, '%Y-%m-%d 00:00:00') as date, count(*) as count FROM screenshots GROUP BY date_format(timestamp, '%Y-%m-%d 00:00:00');")

hv['movies']['date'] = pd.to_datetime(hv['movies']['date'])
hv['movies'] = hv['movies'].sort_values(['date']).reset_index(drop=True)

hv['screenshots']['date'] = pd.to_datetime(hv['screenshots']['date'])
hv['screenshots'] = hv['screenshots'].sort_values(['date']).reset_index(drop=True)
hv['screenshots']

In [ ]:
def service_pause(p, df):
#     box = BoxAnnotation(left=pd.Timestamp('2012/01/01'), right=pd.Timestamp('2013/01/01'), fill_alpha=0.1, fill_color='red', legend='HV')
    p.harea(y=range(0, df['count'].max()), x1=pd.Timestamp('2011/06/07'), x2=pd.Timestamp('2011/06/08'), fill_color='red', fill_alpha=1, legend= "failed eruption (2011/06/07)")
    p.harea(y=range(0, df['count'].max()), x1=pd.Timestamp('2013/11/28'), x2=pd.Timestamp('2013/11/29'), fill_color='purple', fill_alpha=1, legend= "Comet ISON (2013/11/28)")
    p.harea(y=range(0, df['count'].max()), x1=pd.Timestamp('2017/09/06'), x2=pd.Timestamp('2017/09/10'), fill_color='purple', fill_alpha=1, legend= "large flares (2017/9/06-09)")
    p.harea(y=range(0, df['count'].max()), x1=pd.Timestamp('2011/08/11'), x2=pd.Timestamp('2011/09/18'), fill_color='gray', fill_alpha=0.3, legend= "GSFC server repair (2011/08/11 - 2011/09/18)")
    p.harea(y=range(0, df['count'].max()), x1=pd.Timestamp('2015/02/04'), x2=pd.Timestamp('2015/09/23'), fill_color='red', fill_alpha=0.1, legend= "GSFC server down (2015/02/04 - 2015/09/23)")
    p.harea(y=range(0, df['count'].max()), x1=pd.Timestamp('2013/10/01'), x2=pd.Timestamp('2013/10/16'), fill_color='green', fill_alpha=0.3, legend= "U.S. Fed. Gov. shutdown (2013/10/01 - 2013/10/16)")

In [ ]:
for key in hv.keys():
    df = hv[key].copy()
    df = df.set_index('date')
    df = df.reindex(pd.date_range(df.index.min(), df.index.max(), freq='D').to_period('D').to_timestamp(),
                                  fill_value=0)
    df['date'] = df.index
    df = df.reset_index(drop=True)


    TOOLS = "save, pan, box_zoom, reset, wheel_zoom"

    df_src = ColumnDataSource(df)

    p = figure(plot_height=250, x_axis_type="datetime", 
               tools=TOOLS,
               sizing_mode="scale_width", min_border_left = 0)
    
#     p.min_border_left = 0
#     p.min_border_right = 0
#     p.min_border_top = 0

    p.add_layout(Title(text = "Number of %s generated every day"%key, text_font_size = "16pt", text_font_style="bold"), 
                 place = 'above')
    p.add_layout(Title(text = "Date Range: %s - %s"%(df['date'].min().strftime('%Y, %b %d'),df['date'].max().strftime('%Y, %b %d'))), 
                 place = 'above')
    p.add_layout(Title(text="Total {} generated: {:,} | Total Days: {:,} ".format(key,df['count'].sum(), len(df)), text_font_style="italic"), 
                 place = 'above')

    p.background_fill_color="#f5f5f5"
    p.grid.grid_line_color="white"
    p.xaxis.axis_label = 'Date'
    p.yaxis.axis_label = 'No. of %s'%key
    p.axis.axis_line_color = None
    p.x_range.range_padding = 0.02
    p.x_range.range_padding = 0.02
    p.y_range.range_padding = 0.02
    p.yaxis[0].formatter = NumeralTickFormatter(format='0,0')
#     p.xaxis[0].formatter = DatetimeTickFormatter(days=["%b %d, %Y %H"])
    p.xaxis.ticker = YearsTicker(desired_num_ticks=10, num_minor_ticks=12)
#     p.xaxis[0].ticker.desired_num_ticks = 10
    
    p.line(x='date', y='count', line_width=2, color='#ebbd5b', source=df_src)

    p.add_tools(HoverTool(
        tooltips=[( 'date',   '@date{%F}'),
    #               ( 'close',  '$@{adj close}{%0.2f}' ), # use @{ } for field names with spaces
                  ( 'count', '@count{0,0}'),#{0.00 a}'      ),
                 ],
        formatters={'date'      : 'datetime', # use 'datetime' formatter for 'date' field
    #                 'count' : 'int',   # use 'printf' formatter for 'adj close' field
                                      # use default 'numeral' formatter for other fields
                   },
    #     display a tooltip whenever the cursor is vertically in line with a glyph
    #     mode='vline'
    ))
    df_stats = pd.DataFrame({'height': pd.date_range(df['date'].min(),df['date'].max(),periods=2),
                                 'mean':np.nanmean(df['count']), 'median': np.nanmedian(df['count']), 'mode':stats.mode(df['count'])[0][0]})
    p.line(y='mean', x='height', line_color="black", line_dash='dotdash', line_width = 2, legend="Mean (%.2f)"%(df_stats['mean'][0]), source=df_stats)
    p.line(y='median', x='height', line_color = "red", line_dash='dashed', line_width=2, legend="Median (%.2f)"%(df_stats['median'][0]), source=df_stats)
    service_pause(p, df)
    show(p)
    save(p, filename='./%s/time_series.html'%key, title='Helioviewer %s generated every day'%key)

# Histogram of media per day

In [ ]:
def bin_width(m):
    n = np.int(np.log10(m+1))
    n = 10**(n-1)
    q = np.ceil(m/(n*(36))).astype(int)
    bw = max(q*n,1)
    return bw#, m//n+1

for key in hv.keys():
    df = hv[key].copy()
    df = df.set_index('date')
    df = df.reindex(pd.date_range(df.index.min(), df.index.max(), freq='D').to_period('D').to_timestamp(),
                                  fill_value=0)
    df['date'] = df.index
    df = df.reset_index(drop=True)

    bin_size = bin_width(df['count'].max())# np.arange(0,count.max(),) 30#.astype(int)#100
    counts = df['count']

    arr_hist, edges = np.histogram(counts, bins=np.arange(0, counts.max()+bin_size, bin_size))
    cum_bin_size  = max(bin_size//10,1)
    cum_hist, cum_edges, patches = plt.hist(counts, bins=np.arange(0,counts.max()+cum_bin_size, cum_bin_size), cumulative=True)
    plt.close()

    # Column data source
    df_hist = pd.DataFrame({'count': arr_hist, 'left': edges[:-1], 'right': edges[1:]})
    total = df_hist['count'].sum()
    df_hist['f_count'] = ['%d' % count for count in df_hist['count']]
    df_hist['f_percent'] = ['%.2f%%' %(count/total*100) for count in df_hist['count']]
    df_hist['f_interval'] = ['[{:,.0f} - {:,.0f})'.format(left, right) for left, right in zip(df_hist['left'], df_hist['right'])]
    hist_src = ColumnDataSource(df_hist)

    #cumulative data
    cumulative_data = cum_hist#np.cumsum(arr_hist)
    x_bins = cum_edges[1:]#edges[1:]# np.arange(0, counts.max(), bin_size)[1:]
    df_cum = pd.DataFrame({'count_cum': cumulative_data, 'x': x_bins})
    df_cum['f_percent'] = ['%.2f%%' %(count/total*100) for count in df_cum['count_cum']]
    cum_src = ColumnDataSource(df_cum)

    panels = []
    for axis_type in ["log","linear"]:
        p = figure(y_axis_type = axis_type,
                   x_axis_label = 'No. of %s'%key, y_axis_label = 'Day count', 
                   background_fill_color="#fafafa",
                   y_range = (0.9, df_hist['count'].max() + df_hist['count'].max()//10))

        # Add a quad glyph with source this time
        p.quad(bottom=0.9, top='count', left='left', right='right', source=hist_src, fill_color='navy', alpha=0.5,
               hover_fill_color='navy', hover_fill_alpha=0.2, line_color='white', legend='Histogram')
    #         p.y_range(Range1d(0.8,df_hist['count'].max()))
        # Add style to the plot
        p.title.align = 'center'
        p.title.text_font_size = '18pt'
        p.xaxis.axis_label_text_font_size = '12pt'
        p.xaxis.major_label_text_font_size = '12pt'
        p.yaxis.axis_label_text_font_size = '12pt'
        p.yaxis.major_label_text_font_size = '12pt'
        p.yaxis[0].formatter = NumeralTickFormatter(format='0,0')
        p.xaxis[0].formatter = NumeralTickFormatter(format='0,0')
        
        df_stats = pd.DataFrame({'height': np.linspace(0.5, df_hist['count'].max(), 2),
                                 'mean':np.nanmean(counts), 'median': np.nanmedian(counts), 'mode':stats.mode(counts)[0][0]})
        p.line(x='mean', y='height', line_color="black", line_dash='solid', line_width = 4, legend="Mean (%.2f)"%(df_stats['mean'][0]), source=df_stats)
        p.line(x='median', y='height', line_color = "red", line_dash='dashed', line_width=3, legend="Median (%.2f)"%(df_stats['median'][0]), source=df_stats)
#         p.line(x='mode', y='height', line_color = "lightgreen", line_dash = 'dashdot',line_width=2,legend="Mode (%.2f)"%(df_stats['mode'][0]), source=df_stats)
        
        total_days = (counts>=0).sum()
        total_files = counts.sum()

        p.add_layout(Title(text = "Histogram for %s generated per day"%key, text_font_size = "16pt", text_font_style="bold"), place = 'above')
        p.add_layout(Title(text="Date range: %s - %s"%(df['date'].min().strftime('%Y, %b %d'),df['date'].max().strftime('%Y, %b %d'))), 'above')
        p.add_layout(Title(text="Total {} generated: {:,} | Total Days: {:,} ".format(key, total_files, total_days), text_font_style="italic"), 'above')

        p.legend.location = "top_right"
    #     p.grid.grid_line_color="white"

    #     text_source = ColumnDataSource(dict(x=[x_bins.max()*3/4],y=[df_hist['count'].max()*3/4],text=['Total Day Count = \n %d'%total]))
    #     glyph = Text(x="x", y="y", text="text", text_color="black")
    #     p.add_glyph(text_source, glyph)

        # Add a hover tool referring to the formatted columns
        hover = HoverTool(tooltips = [('#%s generated'%key, '@f_interval'),
                                      ('Day count', '@f_count{0,0}'),
                                      ('Day count percentage', '@f_percent')],
                          mode= 'vline')

        # Add the hover tool to the graph
        p.add_tools(hover)
        p2 = figure(y_axis_type=axis_type,
                           x_axis_label = 'No. of %s'%key, 
                           y_axis_label = 'Day count',
                           background_fill_color="#fafafa")


        p2_line = p2.line(x='x', y='count_cum', line_color='#036564', line_width=3, source=cum_src, legend="Cumulative distribution")
    #         p2_circle = p2.circle(x='x', y='count_cum', line_color='#036564', line_width=5, source=cum_src, hover_line_alpha=0.5, legend="Cumulative distribution" )
        p2.add_layout(Title(text = "Cumulative distribution for number of %s generated"%key, text_font_size = "16pt", text_font_style="bold"), place = 'above')
        p2.add_layout(Title(text="Date range: %s - %s"%(df['date'].min().strftime('%Y, %b %d'),df['date'].max().strftime('%Y, %b %d'))), 'above')
        p2.add_layout(Title(text="Total {} generated: {:,} | Total Days: {:,}".format(key, total_files, total_days), text_font_style="italic"), 'above')
    #             p2.add_layout(Title(text="Total Days: %d"%total_days, text_font_style="italic"), 'above')
    #             p2.add_layout(Title(text="Source ID: %d"%sid, text_font_style="italic"), 'above')

        hover = HoverTool(line_policy='nearest', 
                          tooltips = [('#%s generated'%key, '<@x{0,0}'), 
                                      ('Percentage of %s generated'%key, '<@f_percent'),
                                      ('Cumulative Day count', '@count_cum{0,0}')],
                          mode='vline')

    #         p2.add_layout(Span(location=1800, dimension='height'))#, legend='Expected date file count'))
        
        df_cumstats = pd.DataFrame({'height': np.linspace(df_cum['count_cum'].min(),df_cum['count_cum'].max(),2),
                                 'mean':np.nanmean(counts), 'median': np.nanmedian(counts), 'mode':stats.mode(counts)[0][0]})
        p2.line(x='mean', y='height', line_color="black", line_dash='solid', line_width = 4, legend="Mean (%.2f)"%(df_cumstats['mean'][0]), source=df_cumstats)
        p2.line(x='median', y='height', line_color = "red", line_dash='dashed', line_width=3, legend="Median (%.2f)"%(df_cumstats['median'][0]), source=df_cumstats)
#         p2.line(x='mode', y='height', line_color = "lightgreen", line_dash = 'dashdot',line_width=2,legend="Mode (%.2f)"%(df_cumstats['mode'][0]), source=df_cumstats)
        
        # Add the hover tool to the graph
        p2.add_tools(hover)
        p2.title.align = 'center'
        p2.title.text_font_size = '18pt'
        p2.xaxis.axis_label_text_font_size = '12pt'
        p2.xaxis.major_label_text_font_size = '12pt'
        p2.yaxis.axis_label_text_font_size = '12pt'
        p2.yaxis.major_label_text_font_size = '12pt'
        p2.legend.location = "bottom_right"
        p2.yaxis[0].formatter = NumeralTickFormatter(format='0,0')
        p2.xaxis[0].formatter = NumeralTickFormatter(format='0,0')


        grid = gridplot([[p, p2]], sizing_mode='stretch_both')# width_policy='max', height_policy='max')#,plot_width=1200, plot_height=1000, sizing_mode='scale_width')#, plot_width=250, plot_height=250)
        panel = Panel(child=grid, title=axis_type)
    #     panel = Panel(child=p, title=axis_type)
        panels.append(panel)
    tabs = Tabs(tabs=panels)
    show(tabs)
    save(tabs, filename='./%s/histogram.html'%key, title='Histogram for Helioviewer %s generated every day'%key)

# Weekday frequency distribution

In [ ]:
for key in hv.keys():
    df = hv[key].copy()
    df['weekday'] = df['date'].dt.weekday_name
    weekdays = [ 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    df = df.groupby('weekday').sum().reindex(weekdays)
    df['weekday'] = df.index
    df = df.reset_index(drop=True)
    df

    # Column data source
    df['percent'] = np.float64(["%.2f"%(count/df['count'].sum()*100) for count in df['count']])
    df['percent%'] = df['percent'].astype(str)+"%"
    df['vbar_top'] = df['count'].astype(str) + '\n' + df['percent'].astype(str)+'%'
    df_src = ColumnDataSource(df)
    panels = []
    for axis_type in ["linear","log"]:
        p = figure(x_range = df['weekday'],
                   y_axis_type = axis_type,
                   x_axis_label = 'Weekdays', y_axis_label = 'Day count', 
                   background_fill_color="#fafafa", aspect_ratio=16/9, plot_width=1000)

        # Add a quad glyph with source this time
        p.vbar(x='weekday', top='count', width=0.75, source=df_src, bottom=0.1,
               hover_fill_alpha=0.5, line_color='white', legend="weekday",
               fill_color=factor_cmap('weekday', palette=bp.Spectral7, factors=df['weekday']),
               hover_fill_color=factor_cmap('weekday', palette=bp.Spectral7, factors=df['weekday']), 
              )
        # Add style to the plot
        p.title.align = 'center'
        p.title.text_font_size = '18pt'
        p.xaxis.axis_label_text_font_size = '12pt'
        p.xaxis.major_label_text_font_size = '12pt'
        p.yaxis.axis_label_text_font_size = '12pt'
        p.yaxis.major_label_text_font_size = '12pt'
        p.xgrid.grid_line_color = None
        p.y_range.start = 0.1
        p.y_range.end = df['count'].max()*1.5
        p.yaxis[0].formatter = NumeralTickFormatter(format='0,0')
        
        if(axis_type=="log"): p.y_range.end = df['count'].max()**1.5

        p.add_layout(Title(text = "Frequency of %s generated per weekday"%key, text_font_size = "16pt", text_font_style="bold"), place = 'above')
        p.add_layout(Title(text="Date range: %s - %s"%(hv[key]['date'].min().strftime('%Y, %b %d'),hv[key]['date'].max().strftime('%Y, %b %d'))), 'above')
        p.add_layout(Title(text="Total {} generated: {:,} | Total Days: {:,}".format(key,df['count'].sum(), total_days), text_font_style="italic"), 'above')

        p.legend.orientation = "horizontal"
        p.legend.location = "top_center"
        p.grid.grid_line_color="white"
        labels = LabelSet(x='weekday', y='count', text='percent%', level='glyph',
                          x_offset=-30, y_offset=0, source=df_src, render_mode='canvas')
        p.add_layout(labels)

        # Add a hover tool referring to the formatted column

        hover = HoverTool(tooltips = [('%s generated'%key, '@count{0,0}'),
                                      ('Percentage of %s generated'%key, '@percent%')],
                          mode= 'vline')

    #     Add the hover tool to the graph
        p.add_tools(hover)
        panel = Panel(child=p, title=axis_type)
        panels.append(panel)
    tabs = Tabs(tabs=panels)
    show(tabs)
    save(tabs, filename='./%s/weekday_freq.html'%key, title='Histogram for Helioviewer %s generated every day'%key)

# Weekday frequency against week number

In [ ]:
df_service = pd.concat([pd.DataFrame({'date': pd.date_range('2011/08/11', '2011/09/18'), 'reason':"GSFC server repair \n (2011/08/11 - 2011/09/18)"}),
                        pd.DataFrame({'date': pd.date_range('2013/10/01', '2013/10/16'), 'reason':"U.S. Fed. Gov. shutdown \n  (2013/10/01 - 2013/10/16)"}),
                        pd.DataFrame({'date': pd.date_range('2015/02/04', '2015/09/23'), 'reason':"GSFC server down   \n (2015/02/04 - 2015/09/23)"})],
                       ignore_index=True)
df_service['weekday'] = df_service['date'].dt.weekday_name
df_service

In [ ]:
for key in hv.keys():
    df = hv[key].copy()
    df['weekday'] = df['date'].dt.weekday_name
    df['weeknumber'] = ((df['date']-df['date'][0]).dt.days/7).astype(int)
    df_service['weeknumber'] = ((df_service['date']-df['date'][0]).dt.days/7).astype(int)
    # df = df.groupby(['weeknumber','weekday']).sum().reset_index()

    weeknumber = np.array(df['weeknumber'].unique()).astype(str)# hv_cov.index.values#.astype(str)
    # weekdays = weekdays# df['weekday'].unique().astype(str) # np.arange(1,32).astype(str)

    colors = bp.Viridis[256]# ["#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce", "#ddb7b1", "#cc7878", "#933b41", "#550b1d"]

    TOOLS = "save,pan,box_zoom,reset,wheel_zoom"

    # output_file('AIA1600_coverage.html')
    panels = []
    for mapper_type, mapper, ticker in zip(["log", "linear"],
                                           [LogColorMapper, LinearColorMapper],
                                           [LogTicker, BasicTicker]):
        p = figure(#x_axis_type='datetime',
                   x_range=weeknumber, y_range=list(reversed(weekdays)),
                   x_axis_location=None, sizing_mode='stretch_both',# width_policy='max', height_policy='max',#, 
    #                plot_width=2000,
                   x_axis_label="Weeks since first data", y_axis_label="Weekday",
                   tools=TOOLS)

        p_rect = p.rect(x="weeknumber", y="weekday", width=1, height=1,
               source=df,
               color={'field': 'count', 'transform': mapper(palette=colors, low=0.1, high=np.nanmax(df['count']))},
               hover_fill_alpha=0.2)

        p.add_tools(HoverTool(renderers = [p_rect], 
                              tooltips=[('Week Number', '@weeknumber'), 
                                        ('#%s generated'%key, '@count{0,0}'), 
                                        ('Date','@date{%F}')],
                              formatters={'date': 'datetime'}
        ))
        xaxis = LinearAxis(ticker=SingleIntervalTicker(interval=7, num_minor_ticks= 1))
        p.add_layout(xaxis, 'above')
        p.xaxis.axis_label = "Weeks since first data"
        # p.grid.grid_line_color = None
        p.axis.axis_line_color = None
        p.axis.major_tick_line_color = None
        p.axis.major_label_text_font_size = "7px"
        p.axis.major_label_standoff = 0
        p.xaxis.major_label_orientation = np.pi / 3
        p.xaxis.axis_label_text_font_size = "12pt"
    #     p.xaxis.major_label_text_color = {'field': 'weeknumber', 'transform': mapper(palette=bp.Spectral6, low=0.1, high=np.nanmax(df['count']))}
        p.yaxis.axis_label_text_font_size = "12pt"
        p.xaxis.visible = True
        p.xgrid.visible = False
        p.ygrid.visible = False

        p.xaxis.major_label_text_font_size = "7pt"
        p.yaxis.major_label_text_font_size = "8pt"

        p.add_layout(Title(text = "Weekday frequency against week number", text_font_size = "16pt", text_font_style="bold"), place = 'above')
        p.add_layout(Title(text="Date range: %s - %s"%(hv[key]['date'].min().strftime('%Y, %b %d'),hv[key]['date'].max().strftime('%Y, %b %d'))), 'above')
        p.add_layout(Title(text="Total {} generated: {:,} | Total days: {:,} ".format(key,df['count'].sum(), len(df)), text_font_style="italic"), 'above')
        
        p_service = p.rect(x="weeknumber", y="weekday", width=1, height=1,
                           source=df_service,
                           fill_color='red', fill_alpha=0.5, hover_fill_alpha=0.2, line_color=None)
        p.add_tools(HoverTool(renderers = [p_service], 
                              tooltips=[('Week Number', '@weeknumber'), 
                                        ('Shutdown', '@reason'), 
                                        ('Date','@date{%F}')],
                              formatters={'date': 'datetime'}))

        num_ticks=10
        if (len(df[df['count']>0]['count'].unique()) <= 10):
            num_ticks = len(df[df['count']>0]['count'].unique())
        color_bar = ColorBar(color_mapper = mapper(palette=colors, low=0.1, high=np.nanmax(df['count'])), 
                             major_label_text_font_size="10px",
                             ticker=ticker(desired_num_ticks=num_ticks),
                             formatter=PrintfTickFormatter(format="%d"),
                             label_standoff=6, border_line_color=None, location=(0, 0))
        p.add_layout(color_bar, 'right')
    #             p.width_policy = 'fit'
    #             p.height_policy = 'fit'
        panel = Panel(child=p, title=mapper_type)
        panels.append(panel)
    tabs = Tabs(tabs=panels)
    show(tabs)
    save(tabs, filename='./%s/weeknumber_frequency.html'%key, title='Weekday frequency against weeknumber for %s'%key)

In [ ]:
TOOLS = "save, pan, box_zoom, reset, wheel_zoom"
for key in hv.keys():
    df = hv[key].copy()
    df = df.set_index('date')
    df = df.reindex(pd.date_range(df.index.min(), df.index.max(), freq='D').to_period('D').to_timestamp(),
                                  fill_value=0)
    df['date'] = df.index
    df = df.reset_index(drop=True)
    
    df['weekday'] = df['date'].dt.weekday_name
    df['weeknumber'] = ((df['date']-df['date'][0]).dt.days/7).astype(int)
    df_service['weeknumber'] = ((df_service['date']-df['date'][0]).dt.days/7).astype(int)
    # df = df.groupby(['weeknumber','weekday']).sum().reset_index()
    weeknumber = np.array(df['weeknumber'].unique())# hv_cov.index.values#.astype(str)
   
    color = {weekdays[i]:bp.Spectral7[i] for i in range(len(weekdays))}
    p_wd=[]
    panels=[]
    for wd in weekdays:
        df_wd = df.loc[df['weekday']==wd]
        p = figure(x_axis_type="datetime",tools=TOOLS,
                   plot_height=200, sizing_mode="scale_width", 
                   title="Weekly coverage of %s for %s"%(key, wd),
                   x_axis_label="Date", 
                   y_axis_label="Count")
        
        p.title.text_font_size = '16pt'
        p.line(x='date', y='count', source=df_wd, legend=wd, color="red")
        p.add_tools(HoverTool(tooltips=[('Week Number', '@weeknumber'), 
                                        ('Date','@date{%F}'),
                                        ('count','@count')],
                              formatters={'date': 'datetime'},
                              mode='vline'))
        service_pause(p, df_wd)
        panel=Panel(child = p, title=wd)
        panels.append(panel)
    tabs=Tabs(tabs=panels)
#     grid = gridplot(list(np.array([p_wd]).T), sizing_mode="scale_width")
    show(tabs)
    save(tabs, filename='%s/weekly_weekday.html'%key, title='Weekly coverage of movies per weekday')

In [ ]:
df_wd.reset_index(drop=True)

In [ ]:
for wd in ['Monday']:
    df_wd = df.loc[df['weekday']==wd]
    plt.figure()
    plt.subplot(211)
    plt.plot(df_wd['weeknumber'], df_wd['count'], label=wd)
    plt.subplot(212)
    freq = np.fft.fftfreq(df_wd['weeknumber'].size,d=1)[:df_wd['weeknumber'].size//2]
    ft = np.fft.fft(df_wd['count'])[:df_wd['weeknumber'].size//2]/df_wd['count'].size
    df_ft = pd.DataFrame()
    df_ft['freq'] = freq
    df_ft['ft'] = abs(ft)
    plt.plot(df_ft['freq'], df_ft['ft'])
    plt.ylim(0,50)

In [ ]:
1/df_ft.loc[df_ft['freq']>0.01].sort_values('ft')['freq'].iloc[-1]

In [ ]:
x = np.array(df_wd['weeknumber'])
dt = x[1]-x[0]
nf = int(x.size/2+1)
f = np.arange(0,nf)
f = f/(dt*x.size)
f

# Popularity

In [4]:
json_url = urllib.request.urlopen('https://api.helioviewer.org/?action=getDataSources')
hv_keys = json.loads(json_url.read())

In [5]:
hv_sid = pd.DataFrame(columns=['OBS','SOURCE_ID'])

# while sid=='sourceId':
#     key=hv_keys.keys()

for key1 in hv_keys.keys():
    for key2 in hv_keys[key1].keys():
        if 'sourceId' in hv_keys[key1][key2].keys(): 
            hv_sid.loc[len(hv_sid)] = " ".join([key1, key2]), hv_keys[key1][key2]['sourceId']
        else:
            for key3 in hv_keys[key1][key2].keys():
                if 'sourceId' in hv_keys[key1][key2][key3].keys(): 
                    hv_sid.loc[len(hv_sid)] = " ".join([key1, key2, key3]), hv_keys[key1][key2][key3]['sourceId']
                else:
                    for key4 in hv_keys[key1][key2][key3].keys():
                        if 'sourceId' in hv_keys[key1][key2][key3][key4].keys(): 
                            hv_sid.loc[len(hv_sid)] = " ".join([key1, key2, key3, key4]), hv_keys[key1][key2][key3][key4]['sourceId']
                        else:
                            for key5 in hv_keys[key1][key2][key3][key4].keys():
                                if 'sourceId' in hv_keys[key1][key2][key3][key4][key5].keys(): 
                                    hv_sid.loc[len(hv_sid)] = " ".join([key1, key2, key3, key4, key5]), hv_keys[key1][key2][key3][key4][key5]['sourceId']
                                else:
                                    for key6 in hv_keys[key1][key2][key3][key4][key5].keys():
                                        if 'sourceId' in hv_keys[key1][key2][key3][key4][key5][key6].keys(): 
                                            hv_sid.loc[len(hv_sid)] = " ".join([key1, key2, key3, key4, key5,key6]), hv_keys[key1][key2][key3][key4][key5][key6]['sourceId']    

In [6]:
hv_sid = hv_sid.sort_values(['SOURCE_ID']).reset_index(drop=True)
hv_sid

,OBS,SOURCE_ID
0,SOHO EIT 171,0
1,SOHO EIT 195,1
2,SOHO EIT 284,2
3,SOHO EIT 304,3
4,SOHO LASCO C2 white-light,4
...,...,...
72,Hinode XRT Al_poly Any,10009
73,Hinode XRT Be_med Any,10010
74,Hinode XRT Be_thin Any,10011
75,Hinode XRT C_poly Any,10012


In [8]:
sourceId=15

In [34]:
hv={}

In [9]:
hv['data'] = sql_query("SELECT date FROM data FORCE INDEX (date_index) WHERE sourceId={};".format(sourceId))

In [36]:
hv['data']['SOURCE_ID'] = sourceId
hv['data']

,date,SOURCE_ID
0,2010-06-02 00:05:30,15
1,2010-06-02 00:05:54,15
2,2010-06-02 00:06:18,15
3,2010-06-23 00:00:17,15
4,2010-06-23 00:00:41,15
...,...,...
7056347,2020-08-03 13:59:26,15
7056348,2020-08-03 14:00:14,15
7056349,2020-08-03 14:01:02,15
7056350,2020-08-03 14:01:50,15


In [61]:
start_time=time.time()
hv['movies'] = sql_query("SELECT startDate, endDate FROM movies WHERE dataSourceString LIKE \'%SDO%AIA%1600%\';")
print(time.time()-start_time)

1.620729684829712


In [62]:
hv['movies']

,startDate,endDate
0,2011-05-26 20:27:05,2011-05-27 07:49:29
1,2011-05-15 11:32:41,2011-05-16 11:27:05
2,2011-05-15 11:32:41,2011-05-16 11:27:05
3,2011-05-15 11:32:41,2011-05-16 11:27:05
4,2011-05-28 20:00:17,2011-05-28 22:59:05
...,...,...
13297,2020-08-18 13:19:26,2020-08-18 14:19:26
13298,2014-10-25 16:40:16,2014-10-25 18:39:28
13299,2014-10-25 16:40:14,2014-10-25 18:39:29
13300,2020-08-20 21:06:59,2020-08-21 08:22:35


In [33]:
hv_sid.loc[hv_sid['SOURCE_ID']==sourceId]['OBS'].values[0]

'SDO AIA 1600'

In [ ]:
def sql_hv(sourceId, obs=None):
    query = "SELECT date_format(date, '%Y-%m-%d 00:00:00') as date, count(*) as count FROM data FORCE INDEX (date_index) WHERE sourceId={} GROUP BY date_format(date, '%Y-%m-%d 00:00:00');".format(sourceId)
    hv = sql_query(query)
    return hv_prepare(hv, sourceId, obs)

par = Parallel(n_jobs=20)
start_time=time.time()
results = par(delayed(sql_hv)(df['SOURCE_ID'], df['OBS']) for ind, df in hv_sid.iterrows())
print(time.time()-start_time)